In [1]:
from library.file_manager   import read_cvs, read_json, write_json
from library.mail_manager   import create_and_send_emails
from library.basic_utils    import find_in_array_dict, unique_array_dict

In [2]:
def remove_sent_emails(all:list,sent:list,query_keys:list=['_id'])->list:
    unsent      = list()
    sent_query  = [e['fields'] for e in sent]
    for a in all:
        query = {}
        for k in query_keys:
            query[k] = a['fields'][k]
        if find_in_array_dict(sent_query,query) == -1:
            unsent.append(a)
    return unsent

def get_unsent_emails(all_db:str, sent_db:str,all_func=read_json,read_func=read_json,query_keys:list=['_id']):

    all     = all_func(all_db)
    sent    = read_func(sent_db)

    unsent  = unique_array_dict(remove_sent_emails(all,sent,query_keys=query_keys),query_keys=query_keys)

    return unsent

In [3]:
# Filenames
FILENAME_ALL = './localhost_manual/demo.json'            
FILENAME_SENT= '.'.join(FILENAME_ALL.split('.')[:-1])+'-SENT.'+FILENAME_ALL.split('.')[-1]
# Custom reciever for the eventual check
C_RECIEVER      = {'To':'josemedeiro@tecnico.ulisboa.pt',
                   'Cc':None,
                   'Bcc':None}

In [4]:
### Session
SERVER      = read_json('server.json')
USER        = read_json('credentials.json')
SESSION     = {**SERVER, **USER}

In [5]:
unsent_emails   = get_unsent_emails(FILENAME_ALL,FILENAME_SENT,all_func=read_json,
                                    query_keys=['_id'])
sent_emails     = read_json(FILENAME_SENT)

# Print information on unsent emails
print(f'There are in total {len(unsent_emails)} emails to send.')

for u in unsent_emails:
    print(u)

There are in total 1 emails to send.
{'metadata': {'From': 'Gaang <gaang@gmail.com>', 'To': 'joao@tecnico.ulisboa.pt', 'Cc': None, 'Bcc': None, 'Reply-To': 'Gaang <gaang@gmail.com>', 'Date': 'default', 'Subject': '[ATLA] Candidatura'}, 'body': 'template_body/demo', 'fields': {'signature': 'template_body/signature', 'name': 'João', 'projects': None, 'teams': 'POWER', 'gender': 'M', '_id': '1'}}


In [6]:
r = input('Proceed? Press "y" or indicate a number of mails to send, if not press any other key.')
if r.lower() == 'y' or r.isdigit():
    if r.isdigit():
        r = int(r)
    else:
        r = None
    new_emails = create_and_send_emails(unsent_emails,SESSION,custom_reciever=C_RECIEVER,N=r)
    sent_emails = sent_emails + new_emails

In [7]:
### Save sent emails
write_json(FILENAME_SENT,sent_emails)

[]